In [ ]:
pip install transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [ ]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 19.5 MB/s eta 0:00:00


In [ ]:
import torch
import time
import requests
from bs4 import BeautifulSoup
from transformers import AutoProcessor, AutoModelForVision2Seq, Blip2Processor
import gradio as gr

def log_to_file(message):
    """Logs messages for debugging."""
    with open("debug_log.txt", "a") as f:
        f.write(message + "\n")

def extract_website_info(url: str, query: str) -> str:
    """Extracts relevant content from a website while ignoring unnecessary elements."""
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers, timeout=5)  # ⏳ Reduced timeout for speed
        if response.status_code != 200:
            return f"Failed to retrieve page: HTTP {response.status_code}"

        soup = BeautifulSoup(response.text, "html.parser")

        # Remove unnecessary elements (ads, scripts, styles, etc.)
        for tag in soup(["script", "style", "header", "footer", "aside", "nav"]):
            tag.decompose()

        # Extract main content
        extracted_text = []
        for tag in soup.find_all(["h1", "h2", "h3", "p", "li"]):
            text = tag.get_text().strip()
            if text and len(text) > 30:  # Filter out short/irrelevant text
                extracted_text.append(text)

        # Join extracted content
        full_text = "\n".join(extracted_text)
        log_to_file(f"Extracted text length: {len(full_text)} characters")

        # Limit input to 1000 characters for better LLM performance
        full_text = full_text[:1000]

        return f"""
        Summarize the key takeaways from the following extracted webpage content in 3-5 sentences. Avoid repetition:
        {full_text}
        """
    except requests.exceptions.Timeout:
        return "Error: Website took too long to respond."
    except requests.exceptions.ConnectionError:
        return "Error: Could not connect to the website."
    except Exception as e:
        return f"Error fetching webpage: {str(e)}"

def generate_response(user_input: str) -> str:
    """Generates response using Proxy lite model."""
    inputs = processor(text=[user_input], return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=150,  # ⬅ Limit response length for faster output
            num_beams=5,  # ⬅ Improves coherence and avoids nonsense
            repetition_penalty=2.0,  # ⬅ Prevents repeating words
            length_penalty=1.0,
            early_stopping=True
        )
    return processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

def gradio_ui(url: str, query: str):
    start_time = time.time()

    # Extract website content
    extraction_start = time.time()
    extracted_info = extract_website_info(url, query)
    extraction_time = time.time() - extraction_start
    log_to_file(f"Extraction Time: {extraction_time:.2f} sec")

    if extraction_time > 10:
        return f"Web extraction took too long ({extraction_time:.2f} sec). Optimization needed."

    # Generate AI response
    llm_start = time.time()
    response = generate_response(extracted_info)
    llm_time = time.time() - llm_start
    log_to_file(f"LLM Response Time: {llm_time:.2f} sec")

    total_time = time.time() - start_time
    log_to_file(f"Total Time Taken: {total_time:.2f} sec")
    return response

# Load Proxy-Lite-3B model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "convergence-ai/proxy-lite-3b"

# Load the processor which includes the tokenizer
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b", trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True)

iface = gr.Interface(
    fn=gradio_ui,
    inputs=[
        gr.Textbox(label="Enter Website URL"),
        gr.Textbox(label="Enter Your Question"),
    ],
    outputs=gr.Textbox(label="Generated Response"),
    title="UI-TARS Web Agent",
    description="Enter a website URL and a question, and the AI will scrape the content and generate an answer."
)

iface.launch(share=True, debug=True)

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8202820d7bbc9dedf2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
